<a href="https://colab.research.google.com/github/alexwhite116/youtube-transcription/blob/main/youtube_transcription.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>